## Dialogue based chatbot

In [1]:
import numpy as np
from typing import Dict, List, Tuple, Optional

## Loading the model

In [2]:
from intent_classification_model import IntentClassificationModel

In [3]:
IntentClassificationModel().predict("payment method k k available cha")

'payment_method'

In [4]:
from entity_recognition import CRFModel

In [5]:
CRFModel().predict("Dharan ma Redmi Note 9 pro 1 ta delivery garidinu")

array(['B-Location', 'O', 'B-Product', 'I-Product', 'I-Product', 'O',
       'B-Quantity', 'O', 'O', 'O'], dtype=object)

In [6]:
import random

class EcommerceBot:
    def __init__(self, intent_model, entity_model):
        self.intent_model = intent_model
        self.entity_model = entity_model
        
        self.order_db = [
            {
                "Id": "ORD12345",
                "status": "Processing"
            },
            {
                "Id": "ORD00000",
                "status": "Shipping"
            }
        ]
        
        self.intent_handlers = {
            'payment_method': self._handle_payment_method,
            'order_status_inquiry': self._handle_order_status,
            'order_product': self._handle_order_product,
            'product_inquiry': self._handle_product_inquiry
        }
        
        # keep the context of the last intent and all the entities
        self.conversation_context = {}
        
    def process_message(self, user_message: str) -> str:
        intent = self._predict_intent(user_message)
        entities = self._extract_entities(user_message)
        
        print(f"Intent: {intent}")
        print(f"Entities: {entities}")
        
        # handling the intent
        if intent in self.intent_handlers:
            response = self.intent_handlers[intent](entities)
        else:
            response = "Maaf garnu hola maile hajurko kura bujhna sakina."
            
        return response
    
    def _predict_intent(self, sentence):
        intent = self.intent_model.predict(sentence)
        return intent
    
    def _extract_entities(self, message: str) -> Dict[str, str]:
        entities = self.entity_model.predict(message)
        return self._format_entities(entities, message.split())
    
    def _format_entities(self, entities: List, sentence: List) -> Dict[str, str]:
        formatted_entities = {
            "Location": None,
            "Product": None,
            "Brand": None,
            "Category": None,
            "Quantity": None,
            "Order_Number": None
        }
        
        for entity, word in zip(entities, sentence):
            # if the entity is present
            if entity in formatted_entities:
                # if it is a single word entity
                formatted_entities[entity] = word
            # if it is a more length word entity
            elif len(entity.split("-")) == 2:
                # check if the entity is the beginning or intermediate
                longer_entity = entity.split("-")
                if longer_entity[0] == "B":
                    formatted_entities[longer_entity[1]] = word
                elif longer_entity[0] == "I":
                    formatted_entities[longer_entity[1]] += " " + word
        # print(formatted_entities)  
        return formatted_entities
    
    def _update_context(self, intent: str, entities: Dict[str, str]):
        self.conversation_context["last_intent"] = intent
        self.conversation_context.update(entities)
        
    def _handle_payment_method(self, entities: Dict[str, str]) -> str:
        response_sample = [
            "Hami esewa, khalti, mobile banking sabei payment method accept garxau.",
            "tapai esewa, khalti ya kunei mobile banking baata payment garna saknu huncha.",
        ]
        return random.sample(response_sample, 1)[0]
    
    def _handle_order_status(self, entities: Dict[str, str]) -> str:
        if entities["Order_Number"] is None:
            return "Kripaya malai hajurko order number dinuhos."
        for orders in self.order_db:
            if orders["Id"] == entities["Order_Number"]:
                status = orders["status"]
                return f"Hajur ko order ko {status} hudei xa."
            
        return "Kripaya sahi order number dinuhos."
    
    def _handle_order_product(self, entities: Dict[str, str]) -> str:
        return "Tapaiko order confirm vayo"
    
    def _handle_product_inquiry(self, entities: Dict[str, str]) -> str:
        return "yo product ko price 1000 ho."

In [7]:
intent_model = IntentClassificationModel()
entity_model = CRFModel()

In [8]:
bot = EcommerceBot(intent_model, entity_model)

In [9]:
if "B-Brand".split("-")[1] in {"Brand": None}:
    print("Hello")

Hello


In [10]:
bot.process_message("Xiaomi ko Redmi Note 9 mobile ko price kati ho Dharan ma 1 ta delivery garidnu ani ORD12345 track garidnu")

Intent: order_product
Entities: {'Location': 'Dharan', 'Product': 'Redmi Note 9', 'Brand': 'Xiaomi', 'Category': 'mobile', 'Quantity': '1', 'Order_Number': 'ORD12345'}


'Tapaiko order confirm vayo'

In [11]:
bot.process_message("mero order ko status bujhna man xa")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Kripaya malai hajurko order number dinuhos.'

In [12]:
bot.process_message("mero order id ORD12345 ho")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': 'ORD12345'}


'Hajur ko order ko Processing hudei xa.'

In [13]:
bot.process_message("mero order id ORD11221 ho")

Intent: order_status_inquiry
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': 'ORD11221'}


'Kripaya sahi order number dinuhos.'

In [15]:
bot.process_message("k k baata pay garna milcha")

Intent: payment_method
Entities: {'Location': None, 'Product': None, 'Brand': None, 'Category': None, 'Quantity': None, 'Order_Number': None}


'Hami esewa, khalti, mobile banking sabei payment method accept garxau.'